<a href="https://colab.research.google.com/github/Sivaskrr/Sivaskrr/blob/main/Assessment_for_DE_Internship_at_DataGrokr_Sivakumar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession
 .builder
 .appName("<app_name>")
 .getOrCreate())

In [ ]:
df_venue = spark.read.load("ipl_venue.csv",
                     format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
df_matches = spark.read.load("ipl_matches.csv",
                     format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
df_ball_by_ball = spark.read.load("ipl_ball_by_ball.csv",
                     format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
df_matches_with_venue = df_matches.join(df_venue,"venue_id")

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df_topeliminator_venue = df_matches_with_venue.where(df_matches_with_venue.eliminator == "Y").groupBy('venue','venue_id').count().sort(desc("count")).show(3)

+--------------------+--------+-----+
|               venue|venue_id|count|
+--------------------+--------+-----+
|Dubai Internation...|      25|    3|
|Sheikh Zayed Stadium|      38|    2|
|Sardar Patel Stad...|      37|    1|
+--------------------+--------+-----+
only showing top 3 rows



In [ ]:
df_ball_by_ball.where(df_ball_by_ball.dismissal_kind == "caught").groupBy("fielder").count().sort(desc("count")).show(1)

+----------+-----+
|   fielder|count|
+----------+-----+
|KD Karthik|  118|
+----------+-----+
only showing top 1 row



In [ ]:
df_highestwkt_dl = df_ball_by_ball.join(df_matches,"match_id")

In [ ]:
df_highestwkt_dl.where(df_highestwkt_dl.method == "D/L").where(df_highestwkt_dl.is_wicket == "1").groupBy("bowler").count().sort(desc("count")).show(1)

+--------+-----+
|  bowler|count|
+--------+-----+
|R Ashwin|    8|
+--------+-----+
only showing top 1 row



In [ ]:
df_nonpwrply = df_ball_by_ball.where(df_ball_by_ball.overs > "6").where(df_ball_by_ball.extra_runs == "0")

In [ ]:
legal=df_ball_by_ball[(~df_ball_by_ball['extras_type'].isin(['wides', 'noballs'])) & (df_ball_by_ball['overs'].between(7,20))]
runs=legal.groupBy('batsman').sum('batsman_runs')
ball=legal.groupby('batsman').count()
balls=ball.withColumnRenamed("batsman","batsman_1")

In [ ]:
joined=balls.join(runs, balls.batsman_1 == runs.batsman, 'inner')
result_4=joined.withColumn("Strike rate", col("sum(batsman_runs)")*100/col("count")).sort('Strike rate',ascending=False)
result_4.select('batsman','Strike rate').show(10)

+------------+------------------+
|     batsman|       Strike rate|
+------------+------------------+
|  B Stanlake|             250.0|
|Kamran Akmal|213.88888888888889|
|  ER Dwivedi|211.11111111111111|
|    Umar Gul|205.26315789473685|
| AC Blizzard|             200.0|
|    RS Sodhi|             200.0|
|    W Jaffer|             188.0|
|   DJM Short|187.09677419354838|
|   LJ Wright|             186.0|
|  AD Russell|181.85185185185185|
+------------+------------------+
only showing top 10 rows



In [ ]:
df = spark.read.csv('ipl_ball_by_ball.csv',header=True)
df1 = spark.read.csv('ipl_matches.csv',header=True)
df2 = spark.read.csv('ipl_venue.csv',header=True)

In [ ]:
df.createOrReplaceTempView("balls")
spark.sql(
"""
select * from balls
"""
)

df1.createOrReplaceTempView("matches")
spark.sql(
"""
select * from matches
"""
)

df2.createOrReplaceTempView("venue")
spark.sql(
"""
select * from venue
"""
)

DataFrame[venue_id: string, venue: string, city: string]

In [ ]:
spark.sql(
"""
select player_of_match,count(player_of_match) from matches
where neutral_venue = 1
group by player_of_match
order by count(player_of_match) desc 
""").show()

+----------------+----------------------+
| player_of_match|count(player_of_match)|
+----------------+----------------------+
|       YK Pathan|                     3|
|      GJ Maxwell|                     3|
|       JP Duminy|                     3|
|       JH Kallis|                     3|
|       ML Hayden|                     2|
|  AB de Villiers|                     2|
|  Sandeep Sharma|                     2|
|    SR Tendulkar|                     2|
|        DR Smith|                     2|
|    Yuvraj Singh|                     2|
|   KC Sangakkara|                     2|
|       RG Sharma|                     2|
|DPMD Jayawardene|                     2|
|  M Muralitharan|                     2|
|       MK Pandey|                     2|
|        SK Raina|                     2|
|     LRPL Taylor|                     2|
|    AC Gilchrist|                     2|
|         A Singh|                     1|
|        V Sehwag|                     1|
+----------------+----------------

In [ ]:
from pyspark.sql.types import IntegerType
df = df.withColumn("is_wicket", df["is_wicket"].cast(IntegerType()))
df = df.withColumn("batsman_runs", df["batsman_runs"].cast(IntegerType()))
df = df.withColumn("extra_runs", df["extra_runs"].cast(IntegerType()))
df = df.withColumn("total_runs", df["total_runs"].cast(IntegerType()))

In [ ]:
player_dismissed=df.groupby('player_dismissed').count()
total_runs=df.groupBy('batsman').sum('batsman_runs')
joined_data=player_dismissed.join(total_runs, player_dismissed.player_dismissed == total_runs.batsman, 'inner')

In [ ]:
result_7=joined_data.withColumn("Batsmen Avg", col("sum(batsman_runs)")/col("count")).sort('Batsmen Avg',ascending=False)
result_7.select('batsman','Batsmen Avg').show(10)

+-------------+------------------+
|      batsman|       Batsmen Avg|
+-------------+------------------+
|   MN van Wyk|55.666666666666664|
|   RD Gaikwad|              51.0|
|     AC Voges|             45.25|
|     KL Rahul| 44.86440677966102|
|      HM Amla| 44.38461538461539|
|Iqbal Abdulla|              44.0|
|    DA Warner| 42.71544715447155|
|  JM Bairstow|41.578947368421055|
|     CH Gayle| 41.13793103448276|
|     MS Dhoni|40.991150442477874|
+-------------+------------------+
only showing top 10 rows



In [ ]:
spark.sql(
"""
select umpire, count(*)
from ((select umpire1 as umpire from matches) union all
      (select umpire2 from matches)
     ) matches
group by umpire
order by count(*) desc ;
""").show()

+--------------------+--------+
|              umpire|count(1)|
+--------------------+--------+
|              S Ravi|     121|
|     HDPK Dharmasena|      94|
|        AK Chaudhary|      87|
|       C Shamshuddin|      82|
|           M Erasmus|      65|
|           CK Nandan|      57|
|         Nitin Menon|      57|
|          SJA Taufel|      55|
|           Asad Rauf|      51|
|         VA Kulkarni|      50|
|        BNJ Oxenford|      48|
|         CB Gaffaney|      47|
|           RJ Tucker|      46|
|         BR Doctrove|      42|
|         RE Koertzen|      41|
|           Aleem Dar|      38|
|           BF Bowden|      37|
|            NJ Llong|      37|
|KN Ananthapadmana...|      33|
|      RK Illingworth|      31|
+--------------------+--------+
only showing top 20 rows



In [ ]:
spark.sql(
"""
select m.match_id,first(v.venue),first(v.city),sum(b.batsman_runs) from balls as b
inner join matches as m on b.match_id=m.match_id
inner join venue as v on m.venue_id =v.venue_id
where b.batsman= 'V Kohli'
group by m.match_id
order by sum(b.batsman_runs) desc limit 1; 
""").show()

+--------+--------------------+-----------+---------------------------------+
|match_id|        first(venue)|first(city)|sum(CAST(batsman_runs AS DOUBLE))|
+--------+--------------------+-----------+---------------------------------+
|  980999|M.Chinnaswamy Sta...|  Bengaluru|                            113.0|
+--------+--------------------+-----------+---------------------------------+



In [ ]:
class Database:
  def __init__(self, appName = "Data"):
    self.spark = (SparkSession
    .builder
    .appName(appName)
    .getOrCreate())
    try:
      self.df_matches = spark.read.load("ipl_matches.csv",format="csv", sep=",", inferSchema="true", header="true")
    except:
      print("Dataset not found - kindly upload dataset to colab notebook")
    self.df_ball_by_ball = spark.read.load("ipl_ball_by_ball.csv",format="csv", sep=",", inferSchema="true", header="true")
    self.df_venue = spark.read.load("ipl_venue.csv",format="csv", sep=",", inferSchema="true", header="true")
    self.df = spark.read.csv('ipl_ball_by_ball.csv',header=True)
    self.df1 = spark.read.csv('ipl_matches.csv',header=True)
    self.df2 = spark.read.csv('ipl_venue.csv',header=True)
  def get_query1_result(self):
    self.df_topeliminator_venue = df_matches_with_venue.where(df_matches_with_venue.eliminator == "Y").groupBy('venue','venue_id').count().sort(desc("count")).show(3)
  
  def get_query2_result(self):
    self.df_ball_by_ball.where(df_ball_by_ball.dismissal_kind == "caught").groupBy("fielder").count().sort(desc("count")).show(1)
  

In [ ]:
db.get_query1_result()

+--------------------+--------+-----+
|               venue|venue_id|count|
+--------------------+--------+-----+
|Dubai Internation...|      25|    3|
|Sheikh Zayed Stadium|      38|    2|
|            Newlands|      33|    1|
+--------------------+--------+-----+
only showing top 3 rows

